# Bild Kompression

In [ ]:
import matplotlib.pyplot as plt
from sklearn.datasets import load_sample_image
plt.rcParams["figure.figsize"] = (10,8)

china = load_sample_image("china.jpg")
ax = plt.axes(xticks=[], yticks=[])
ax.imshow(china);

Das Bild selbst ist als 3-dimensionale Matrix gespeichert (size, height, RGB):

In [ ]:
china.shape

Wir wandeln die Matrix um in [n Samples x m Features], wobei jede Koordinaten ein Sample wir, und die Features die Farben sind.

Die Farben skalieren wir noch auf [0, 1]:

In [ ]:
data = china / 255.0
data = data.reshape(427 * 640, 3)
data.shape

Die "Farbwolke" können wir visualisieren (aus Effizienzgründen ein Subsample von 10.000 Punkten):

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def plot_pixels(data, title, colors=None, N=10000):
    if colors is None:
        colors = data

    # choose a random subset
    rng = np.random.default_rng(0)
    i = rng.permutation(data.shape[0])[:N]
    colors = colors[i]
    R, G, B = data[i].T

    fig, ax = plt.subplots(1, 3, figsize=(16, 6))
    ax[0].scatter(R, G, color=colors, marker='.', alpha=0.1)
    ax[0].set(xlabel='Rot', ylabel='Grün', xlim=(0, 1), ylim=(0, 1))

    ax[1].scatter(R, B, color=colors, marker='.', alpha=0.1)
    ax[1].set(xlabel='Rot', ylabel='Blau', xlim=(0, 1), ylim=(0, 1))

    ax[2].scatter(G, B, color=colors, marker='.', alpha=0.1)
    ax[2].set(xlabel='Grün', ylabel='Blau', xlim=(0, 1), ylim=(0, 1))

    fig.suptitle(title, size=20)

plot_pixels(data, title='Input Farbraum: 16 Millionen mögliche Farben')

Oder als 3-D:

In [ ]:
ax = plt.axes(projection='3d')

rng = np.random.default_rng(0)
i = rng.permutation(data.shape[0])[:10000]
R, G, B = data[i].T
ax.scatter(R, G, B, c=data[i], alpha=0.1);

Jetzt reduzieren wir diese 16 Millionen Farben zu nur noch 16 Farben mittels (MiniBatch) k-Means:

In [ ]:
from sklearn.cluster import MiniBatchKMeans
kmeans = MiniBatchKMeans(16)
kmeans.fit(data)
new_colors = kmeans.cluster_centers_[kmeans.predict(data)]

plot_pixels(data, colors=new_colors,
            title="Reduzierter Farbraum: 16 Farben")

Im Ergebnis ist jedem Pixel der Farbwert eines der 16 des Cluster zugewiesen, diese sind:

In [ ]:
kmeans.cluster_centers_

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
R, G, B = kmeans.cluster_centers_.T
ax = plt.axes(projection='3d')

ax.scatter(R, G, B, c=kmeans.cluster_centers_)
ax.set_title('Die Cluster-Farben', size=16);

Angewandt auf das Bild:

In [ ]:
china_recolored = new_colors.reshape(china.shape)

fig, ax = plt.subplots(1, 2, figsize=(16, 6),
                       subplot_kw=dict(xticks=[], yticks=[]))
fig.subplots_adjust(wspace=0.05)
ax[0].imshow(china)
ax[0].set_title('Original Image', size=16)
ax[1].imshow(china_recolored)
ax[1].set_title('16-Farben Image', size=16);

Kompressionsfaktor: 1 Million

(Natürlich gibt es bessere Kompressionsalgorithmen)